<a href="https://colab.research.google.com/github/gamzef/INF003/blob/master/Endabgabe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#As stated in the description, the data for this competition comes from a experimental set-up used to study earthquake physics. 
#Our goal is to predict the time remaining before the next laboratory earthquake. 
#The only feature we have is the seismic signal (acoustic data), which is recorded using a piezoceramic sensor and corresponds to the voltage upon deformation (in integers).
#Training data: single, continuous segment of experimental data.

#Test data: consists of a folder containing many small segments. The data within each test file is continuous, but the test files do not represent a continuous segment of the experiment.

#Both the training and the testing set come from the same experiment.

#There is no overlap between the training and testing sets.

In [1]:
import numpy as np
import pandas as pd
import os
import gc
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
os.listdir('../content/drive/My Drive/input/')

['sample_submission.csv', 'test']

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
from google.colab import files  #Colab's file access feature
uploaded = files.upload()   #retrieve uploaded file
#upload Kaggle JSON File 
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/kaggle/kaggle.json 

Saving kaggle.json to kaggle.json
chmod: cannot access '/root/kaggle/kaggle.json': No such file or directory


In [5]:
#download earthquake data
!kaggle competitions download -c LANL-Earthquake-Prediction

seg_004cd2.csv: Skipping, found more recently modified local copy (use --force to force download)
seg_00be11.csv: Skipping, found more recently modified local copy (use --force to force download)
seg_004f1f.csv: Skipping, found more recently modified local copy (use --force to force download)
seg_00648a.csv: Skipping, found more recently modified local copy (use --force to force download)
seg_007a37.csv: Skipping, found more recently modified local copy (use --force to force download)
seg_0042cc.csv: Skipping, found more recently modified local copy (use --force to force download)
seg_00a37e.csv: Skipping, found more recently modified local copy (use --force to force download)
seg_003339.csv: Skipping, found more recently modified local copy (use --force to force download)
seg_0125d9.csv: Skipping, found more recently modified local copy (use --force to force download)
seg_00cc91.csv: Skipping, found more recently modified local copy (use --force to force download)
seg_0012b5.csv: Skip

In [6]:
#unzip training data for usage
!unzip train.csv.zip

Archive:  train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               train.csv



In [7]:
#train.csv'deki datayı okur, ilk data tipi acoustic_data, integer, ikincisi time to failure float. 
train_data = pd.read_csv('train.csv', nrows=10, dtype = {'acoustic_data': np.int16, 'time_to_failure': np.float32})

In [8]:
pd.options.display.precision = 15
train_data.head()

,acoustic_data,time_to_failure
0,12,1.469099998474121
1,6,1.469099998474121
2,8,1.469099998474121
3,5,1.469099998474121
4,8,1.469099998474121


In [9]:
#import
train_data = pd.read_csv('train.csv',dtype = {'acoustic_data':np.int16,'time_to_failure':np.float32})

In [10]:
rows = 150000
segments = int(np.floor(train_data.shape[0] / rows))  #6 milyon / 150000 = 4194

X_train = pd.DataFrame(index = range(segments),dtype = np.float32,columns = ['mean','std','99quat','50quat','25quat','1quat'])
y_train = pd.DataFrame(index = range(segments),dtype = np.float32,columns = ['time_to_failure'])

In [11]:
for segment in tqdm(range(segments)):
    x = train_data[segment*rows:segment*rows+rows]
    y = x['time_to_failure'].values[-1]
    x = x['acoustic_data'].values
    X_train.loc[segment,'mean'] = np.mean(x)
    X_train.loc[segment,'std']  = np.std(x)
    X_train.loc[segment,'99quat'] = np.quantile(x,0.99)
    X_train.loc[segment,'50quat'] = np.quantile(x,0.5)
    X_train.loc[segment,'25quat'] = np.quantile(x,0.25)
    X_train.loc[segment,'1quat'] =  np.quantile(x,0.01)
    y_train.loc[segment,'time_to_failure'] = y


100%|██████████| 4194/4194 [00:42<00:00, 99.17it/s]


In [29]:
X_train.head()

,mean,std,99quat,50quat,25quat,1quat
0,4.884113311767578,5.101089000701904,18.0,5.0,3.0,-8.0
1,4.725766658782959,6.588801860809326,21.0,5.0,2.0,-11.0
2,4.906393527984619,6.967373847961426,26.0,5.0,2.0,-15.0
3,4.902239799499512,6.922282218933105,22.0,5.0,2.0,-12.0
4,4.908720016479492,7.301085948944092,26.0,5.0,2.0,-15.0


In [12]:
scaler = StandardScaler()
X_scaler = scaler.fit_transform(X_train)

In [13]:
gc.collect()

33

In [14]:
model = Sequential()
model.add(Dense(32,input_shape = (6,),activation = 'relu'))
model.add(Dense(32,activation = 'relu'))
model.add(Dense(32,activation = 'relu'))
model.add(Dense(1))
model.compile(loss = 'mae',optimizer = 'adam')

In [15]:
model.fit(X_scaler,y_train.values.flatten(),epochs = 50)

Epoch 1/50
4194/4194 [==============================] - 1s 180us/step - loss: 3.4142
Epoch 2/50
4194/4194 [==============================] - 0s 50us/step - loss: 2.2504
Epoch 3/50
4194/4194 [==============================] - 0s 45us/step - loss: 2.2083
Epoch 4/50
4194/4194 [==============================] - 0s 39us/step - loss: 2.1931
Epoch 5/50
4194/4194 [==============================] - 0s 42us/step - loss: 2.1791
Epoch 6/50
4194/4194 [==============================] - 0s 39us/step - loss: 2.1814
Epoch 7/50
4194/4194 [==============================] - 0s 42us/step - loss: 2.1745
Epoch 8/50
4194/4194 [==============================] - 0s 41us/step - loss: 2.1765
Epoch 9/50
4194/4194 [==============================] - 0s 41us/step - loss: 2.1709
Epoch 10/50
4194/4194 [==============================] - 0s 41us/step - loss: 2.1709
Epoch 11/50
4194/4194 [==============================] - 0s 41us/step - loss: 2.1602
Epoch 12/50
4194/4194 [==============================] - 0s 37us/step - l

In [16]:
sub_data = pd.read_csv('../content/drive/My Drive/input/sample_submission.csv',index_col = 'seg_id')

In [17]:
X_test = pd.DataFrame(columns = X_train.columns,dtype = np.float32,index = sub_data.index)

In [19]:
for seq in tqdm(X_test.index):
    test_data = pd.read_csv('../content/drive/My Drive/input/test/'+seq+'.csv')
    x = test_data['acoustic_data'].values
    X_test.loc[seq,'mean'] = np.mean(x)
    X_test.loc[seq,'std']  = np.std(x)
    X_test.loc[seq,'99quat'] = np.quantile(x,0.99)
    X_test.loc[seq,'50quat'] = np.quantile(x,0.5)
    X_test.loc[seq,'25quat'] = np.quantile(x,0.25)
    X_test.loc[seq,'1quat'] =  np.quantile(x,0.01)

100%|██████████| 2624/2624 [14:52<00:00,  2.94it/s]


In [20]:
X_test_scaler = scaler.transform(X_test)

In [21]:
pred = model.predict(X_test_scaler)

In [26]:
sub_data['time_to_failure'] = pred
sub_data['seg_id'] = sub_data.index

In [27]:
sub_data.head()

,time_to_failure,seg_id
seg_id,,
seg_00030f,5.223076820373535,seg_00030f
seg_0012b5,4.500625133514404,seg_0012b5
seg_00184e,3.604608297348022,seg_00184e
seg_003339,8.413659095764160,seg_003339
seg_0042cc,5.600977420806885,seg_0042cc


In [28]:
sub_data.to_csv('sub_earthquake.csv',index = False)